In [70]:
import pandas as pd 
import numpy as np
import warnings 
warnings.filterwarnings('ignore')
from geopy.geocoders import Nominatim
from geopy.exc import GeocoderTimedOut

In [71]:
df = pd.ExcelFile('xlsx\homicidios.xlsx')
df1 = pd.read_excel(df, 'HECHOS')
df2 = pd.read_excel(df,'VICTIMAS')

In [72]:
#Renombramos la columna "ID_HECHO" a "ID"
df2= df2.rename(columns={'ID_hecho': 'ID'})

In [73]:
df_homicidios = df2.merge(df1, on='ID', how='inner' )

In [74]:
# Borramos las columnas: 'FECHA_x', 'AAAA_x' and 2 other columns
df_homicidios = df_homicidios.drop(columns=['FECHA_x', 'AAAA_x', 'MM_x', 'DD_x'])

# Borramos las columnas: 'Altura', 'Cruce', 'Dirección Normalizada'
df_homicidios = df_homicidios.drop(columns=['Altura', 'Cruce', 'Dirección Normalizada'])

# Borramos las columnas: 'AAAA_y', 'MM_y', 'DD_y'
df_homicidios = df_homicidios.drop(columns=['AAAA_y', 'MM_y', 'DD_y'])

In [75]:
 # Renombramos column 'VICTIMA_y' to 'VICTIMA'
df_homicidios = df_homicidios.rename(columns={'VICTIMA_y': 'VICTIMA'})
# Renombramos column 'FECHA_y' to 'FECHA'
df_homicidios = df_homicidios.rename(columns={'FECHA_y': 'FECHA'})

# Capitalizamos el primer carácter de cada dato en nuestras columnas, ordenando sus mayúsculas 

In [76]:
df_homicidios['ID'] = df_homicidios['ID'].str.title()
df_homicidios['ROL'] = df_homicidios['ROL'].str.title()
df_homicidios['VICTIMA_x'] = df_homicidios['VICTIMA_x'].str.title()
df_homicidios['SEXO'] = df_homicidios['SEXO'].str.title()
df_homicidios['LUGAR_DEL_HECHO'] = df_homicidios['LUGAR_DEL_HECHO'].str.title()
df_homicidios['TIPO_DE_CALLE'] = df_homicidios['TIPO_DE_CALLE'].str.title()
df_homicidios['Calle'] = df_homicidios['Calle'].str.title()
df_homicidios['XY (CABA)'] = df_homicidios['XY (CABA)'].str.title()
df_homicidios['pos x'] = df_homicidios['pos x'].str.title()
df_homicidios['pos y'] = df_homicidios['pos y'].str.title()
df_homicidios['PARTICIPANTES'] = df_homicidios['PARTICIPANTES'].str.title()
df_homicidios['ACUSADO'] = df_homicidios['ACUSADO'].str.title()
df_homicidios['ROL'] = df_homicidios['ROL'].str.title()

In [77]:
df_homicidios = df_homicidios[(df_homicidios['PARTICIPANTES'] != 'Sd') & (df_homicidios['VICTIMA'] != 'Sd') & (df_homicidios['ACUSADO'] != 'Sd')]

In [78]:
def clean_data(df_homicidios):
    # Drop column: 'VICTIMA'
    df_homicidios = df_homicidios.drop(columns=['VICTIMA'])
    # Rename column 'VICTIMA_x' to 'VICTIMA'
    df_homicidios = df_homicidios.rename(columns={'VICTIMA_x': 'VICTIMA'})
    return df_homicidios

df_homicidios = clean_data(df_homicidios.copy())
df_homicidios.head(2)

,ID,ROL,VICTIMA,SEXO,EDAD,FECHA_FALLECIMIENTO,N_VICTIMAS,FECHA,HORA,HH,LUGAR_DEL_HECHO,TIPO_DE_CALLE,Calle,COMUNA,XY (CABA),pos x,pos y,PARTICIPANTES,ACUSADO
0,2016-0001,Conductor,Moto,Masculino,19,2016-01-01 00:00:00,1,2016-01-01,04:00:00,4,Av Piedra Buena Y Av Fernandez De La Cruz,Avenida,Piedra Buena Av.,8,Point (98896.78238426 93532.43437792),-58.47533969,-34.68757022,Moto-Auto,Auto
1,2016-0002,Conductor,Auto,Masculino,70,2016-01-02 00:00:00,1,2016-01-02,01:15:00,1,Av Gral Paz Y Av De Los Corrales,Gral Paz,"Paz, Gral. Av.",9,Point (95832.05571093 95505.41641999),-58.50877521,-34.66977709,Auto-Pasajeros,Pasajeros


***Realizamos un cambio de tipos de dato y rellenamos los valores faltantes en FECHA FALLECIMIENTO, considerando que la víctima falleció en el momento ya que se desconoce la fecha exacta***

In [79]:
df_homicidios['FECHA_FALLECIMIENTO'] = df_homicidios['FECHA_FALLECIMIENTO'].astype(str)
df_homicidios['FECHA'] = df_homicidios['FECHA'].astype(str)
df_homicidios.loc[df_homicidios['FECHA_FALLECIMIENTO'] == 'SD', 'FECHA_FALLECIMIENTO'] = df_homicidios['FECHA']
df_homicidios['FECHA_FALLECIMIENTO'] = df_homicidios['FECHA_FALLECIMIENTO'].str.replace('T', ' ')

# Removemos el año de la columna ID

In [80]:
df_homicidios['ID'] = df_homicidios['ID'].str.split('-').str[1]

*Podemos ver que la columna "FECHA_FALLECIMIENTO" posee datos nulos, por lo cual por criterio personal, se consta que la fecha de fallecimiento de la víctima que en "FECHA_FALLECIMIENTO" sea nulo, "FECHA" es la fecha del siniestro*

In [81]:
df_homicidios['FECHA_FALLECIMIENTO'] = df_homicidios['FECHA_FALLECIMIENTO'].fillna(df_homicidios['FECHA'])

In [82]:
# Renombramos la columna
df_homicidios = df_homicidios.rename(columns={'FECHA': 'FECHA_SINIESTRO'})


*Procedemos a ordenar "LUGAR_DEL_HECHO"*

In [83]:
df_homicidios['LUGAR_DEL_HECHO'] =df_homicidios['LUGAR_DEL_HECHO'].str.replace(',', '')

Eliminamos "Calle" que tiene los datos mal ordenados, siendo que "LUGAR_DEL_HECHO" tiene los datos perfectos para nuestro análisis

In [84]:

df_homicidios = df_homicidios.drop(columns=['Calle'])


Eliminamos "XY (CABA)" ya que tenemos las columnas con su correspondiente coordenada "X" e "Y"

In [85]:

df_homicidios = df_homicidios.drop(columns=['XY (CABA)'])


Reemplazamos el nombre "HORA" ya que corresponde a la hora del siniestro

In [86]:

df_homicidios = df_homicidios.rename(columns={'HORA': 'HORA_SINIESTRO'})


In [87]:
# Identificar duplicados
duplicates = df_homicidios.duplicated('ID')

# Reemplazar duplicados con NaN
df_homicidios.loc[duplicates, 'ID'] = np.nan

# Rellenar NaN con el índice correspondiente
df_homicidios['ID'] = df_homicidios['ID'].fillna(df_homicidios.index.to_series())

# Convertir la columna 'ID' a int
df_homicidios['ID'] = df_homicidios['ID'].astype(int)

*Decidimos completar valores faltantes en "EDAD" con la media de todos los valores en esa columna para completar nuestros datos y así obtener un análisis más preciso*

In [88]:
# convertimos la columna 'EDAD' a numérica, ignorando los errores
df_homicidios['EDAD'] = pd.to_numeric(df_homicidios['EDAD'], errors='coerce')
# Calculamos la media de los valores numéricos en la columna 'EDAD'
mean = df_homicidios['EDAD'].mean()
# Reemplazar 'Sd' con la media
df_homicidios['EDAD'] = df_homicidios['EDAD'].fillna(mean)
# convertimos la columna 'EDAD' de nuevo a int
df_homicidios['EDAD'] = df_homicidios['EDAD'].astype(int)

# Realizamos un codigo que integre las fechas del siniestro en semestres para nuestro informe de rendimiento(KPI)

In [89]:
df_homicidios['FECHA_SINIESTRO'] = pd.to_datetime(df_homicidios['FECHA_SINIESTRO'])
df_homicidios['semestre'] = [1 if month <= 6 else 2 for month in df_homicidios['FECHA_SINIESTRO'].dt.month]

realizamos un conteo de nuestros "SinDato" para saber si debemos continuar con el análisis o podemos pasar a la siguiente etapa

In [90]:
conteo_sd = df_homicidios.apply(lambda x: x.value_counts().get('Sd', 0))

In [91]:
conteo_sd

ID                     0
ROL                    6
VICTIMA                4
SEXO                   2
EDAD                   0
FECHA_FALLECIMIENTO    0
N_VICTIMAS             0
FECHA_SINIESTRO        0
HORA_SINIESTRO         0
HH                     0
LUGAR_DEL_HECHO        0
TIPO_DE_CALLE          0
COMUNA                 0
pos x                  0
pos y                  0
PARTICIPANTES          0
ACUSADO                0
semestre               0
dtype: int64

In [92]:
df_homicidios

,ID,ROL,VICTIMA,SEXO,EDAD,FECHA_FALLECIMIENTO,N_VICTIMAS,FECHA_SINIESTRO,HORA_SINIESTRO,HH,LUGAR_DEL_HECHO,TIPO_DE_CALLE,COMUNA,pos x,pos y,PARTICIPANTES,ACUSADO,semestre
0,1,Conductor,Moto,Masculino,19,2016-01-01 00:00:00,1,2016-01-01,04:00:00,4,Av Piedra Buena Y Av Fernandez De La Cruz,Avenida,8,-58.47533969,-34.68757022,Moto-Auto,Auto,1
1,2,Conductor,Auto,Masculino,70,2016-01-02 00:00:00,1,2016-01-02,01:15:00,1,Av Gral Paz Y Av De Los Corrales,Gral Paz,9,-58.50877521,-34.66977709,Auto-Pasajeros,Pasajeros,1
2,3,Conductor,Moto,Masculino,30,2016-01-03 00:00:00,1,2016-01-03,07:00:00,7,Av Entre Rios 2034,Avenida,1,-58.39040293,-34.63189362,Moto-Auto,Auto,1
4,5,Conductor,Moto,Masculino,29,2016-02-01 00:00:00,1,2016-01-21,05:20:00,5,Av San Juan Y Presidente Luis Saenz Peña,Avenida,1,-58.38718297,-34.62246630,Moto-Pasajeros,Pasajeros,1
5,8,Conductor,Moto,Masculino,30,2016-01-24 00:00:00,1,2016-01-24,18:30:00,18,Av 27 De Febrero Y Av Escalada,Avenida,8,-58.44451316,-34.68475866,Moto-Objeto Fijo,Objeto Fijo,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
712,712,Peaton,Peaton,Femenino,50,2021-12-12 00:00:00,1,2021-12-12,06:20:00,6,Av. Rivadavia Y Av. Pueyrredon,Avenida,3,-58.40596860,-34.61011987,Peaton-Auto,Auto,2
713,713,Pasajero_Acompañante,Moto,Femenino,18,2021-12-18 00:00:00,1,2021-12-13,17:10:00,17,Av. Riestra Y Mom,Avenida,7,-58.43353773,-34.64561636,Moto-Auto,Auto,2
714,714,Pasajero_Acompañante,Moto,Femenino,43,2021-12-20 00:00:00,1,2021-12-20,01:10:00,1,Au Dellepiane Y Lacarra,Autopista,9,-58.46739825,-34.65117757,Moto-Auto,Auto,2
715,715,Conductor,Moto,Masculino,27,2022-01-02 00:00:00,1,2021-12-30,00:43:00,0,Av. Gaona Y Terrada,Avenida,11,-58.47293407,-34.61984745,Moto-Cargas,Cargas,2


relleno faltantes en hora

In [93]:
# Extraer las horas y minutos de las cadenas de texto
df_homicidios['HORA'] = df_homicidios['HORA_SINIESTRO'].str.split(':').str[0]
df_homicidios['MINUTO'] = df_homicidios['HORA_SINIESTRO'].str.split(':').str[1]

# Convertir las horas y minutos a números
df_homicidios['HORA'] = pd.to_numeric(df_homicidios['HORA'], errors='coerce')
df_homicidios['MINUTO'] = pd.to_numeric(df_homicidios['MINUTO'], errors='coerce')

# Calcular la media de las horas y minutos que no son NaN
mean_hora = df_homicidios['HORA'].mean()
mean_minuto = df_homicidios['MINUTO'].mean()

# Crear la media de la hora como una cadena de texto
mean_time = f"{int(mean_hora)}:{int(mean_minuto)}"

# Reemplazar "Sd" con la media de la hora
df_homicidios.loc[df_homicidios['HORA_SINIESTRO'] == 'Sd', 'HORA_SINIESTRO'] = mean_time
df_homicidios.loc[df_homicidios['HORA_SINIESTRO'] == 'SD', 'HORA_SINIESTRO'] = mean_time
# Eliminar las columnas 'HORA' y 'MINUTO'
df_homicidios = df_homicidios.drop(columns=['HORA', 'MINUTO'])

reorganizo la direccion


In [94]:
def reorganizar_direcc(address):
    words = address.split()
    if "Av." in words:
        words.remove("Av.")
        words = ["Av."] + words
    return " ".join(words)

df_homicidios['LUGAR_DEL_HECHO'] = df_homicidios['LUGAR_DEL_HECHO'].apply(reorganizar_direcc)

In [95]:
formato_incorrecto = pd.to_datetime(df_homicidios['HORA_SINIESTRO'], errors='coerce').isnull()

# Convertir el formato ISO 8601 a HH:mm:ss
df_homicidios.loc[~formato_incorrecto, 'HORA_SINIESTRO'] = pd.to_datetime(df_homicidios.loc[~formato_incorrecto, 'HORA_SINIESTRO']).dt.strftime('%H:%M:%S')


In [96]:
df_homicidios.loc[df_homicidios['HH'] == 'SD', 'HH'] = mean_time

Eliminamos la columna "HH" ya que representa el entero de las horas, pero lo redondea a la hora dada sin contemplar un redondeo sobre los minutos 

In [97]:
df_homicidios = df_homicidios.drop(columns=['HH'])

Una vez logrado el proceso completo, decidimos guardar nuestro archivo en .csv 

In [98]:
df_homicidios

,ID,ROL,VICTIMA,SEXO,EDAD,FECHA_FALLECIMIENTO,N_VICTIMAS,FECHA_SINIESTRO,HORA_SINIESTRO,LUGAR_DEL_HECHO,TIPO_DE_CALLE,COMUNA,pos x,pos y,PARTICIPANTES,ACUSADO,semestre
0,1,Conductor,Moto,Masculino,19,2016-01-01 00:00:00,1,2016-01-01,04:00:00,Av Piedra Buena Y Av Fernandez De La Cruz,Avenida,8,-58.47533969,-34.68757022,Moto-Auto,Auto,1
1,2,Conductor,Auto,Masculino,70,2016-01-02 00:00:00,1,2016-01-02,01:15:00,Av Gral Paz Y Av De Los Corrales,Gral Paz,9,-58.50877521,-34.66977709,Auto-Pasajeros,Pasajeros,1
2,3,Conductor,Moto,Masculino,30,2016-01-03 00:00:00,1,2016-01-03,07:00:00,Av Entre Rios 2034,Avenida,1,-58.39040293,-34.63189362,Moto-Auto,Auto,1
4,5,Conductor,Moto,Masculino,29,2016-02-01 00:00:00,1,2016-01-21,05:20:00,Av San Juan Y Presidente Luis Saenz Peña,Avenida,1,-58.38718297,-34.62246630,Moto-Pasajeros,Pasajeros,1
5,8,Conductor,Moto,Masculino,30,2016-01-24 00:00:00,1,2016-01-24,18:30:00,Av 27 De Febrero Y Av Escalada,Avenida,8,-58.44451316,-34.68475866,Moto-Objeto Fijo,Objeto Fijo,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
712,712,Peaton,Peaton,Femenino,50,2021-12-12 00:00:00,1,2021-12-12,06:20:00,Av. Rivadavia Y Av. Pueyrredon,Avenida,3,-58.40596860,-34.61011987,Peaton-Auto,Auto,2
713,713,Pasajero_Acompañante,Moto,Femenino,18,2021-12-18 00:00:00,1,2021-12-13,17:10:00,Av. Riestra Y Mom,Avenida,7,-58.43353773,-34.64561636,Moto-Auto,Auto,2
714,714,Pasajero_Acompañante,Moto,Femenino,43,2021-12-20 00:00:00,1,2021-12-20,01:10:00,Au Dellepiane Y Lacarra,Autopista,9,-58.46739825,-34.65117757,Moto-Auto,Auto,2
715,715,Conductor,Moto,Masculino,27,2022-01-02 00:00:00,1,2021-12-30,00:43:00,Av. Gaona Y Terrada,Avenida,11,-58.47293407,-34.61984745,Moto-Cargas,Cargas,2


# *A continuación, utilizaremos la geocodificación para encontrar nuestras coordenadas faltantes, así lograremos contemplar de manera precisa los lugares del accidente faltantes* 

In [99]:
df_homicidios.info()

<class 'pandas.core.frame.DataFrame'>
Index: 694 entries, 0 to 716
Data columns (total 17 columns):
 #   Column               Non-Null Count  Dtype         
---  ------               --------------  -----         
 0   ID                   694 non-null    int32         
 1   ROL                  694 non-null    object        
 2   VICTIMA              694 non-null    object        
 3   SEXO                 694 non-null    object        
 4   EDAD                 694 non-null    int32         
 5   FECHA_FALLECIMIENTO  694 non-null    object        
 6   N_VICTIMAS           694 non-null    int64         
 7   FECHA_SINIESTRO      694 non-null    datetime64[ns]
 8   HORA_SINIESTRO       694 non-null    object        
 9   LUGAR_DEL_HECHO      694 non-null    object        
 10  TIPO_DE_CALLE        694 non-null    object        
 11  COMUNA               694 non-null    int64         
 12  pos x                694 non-null    object        
 13  pos y                694 non-null    obj

In [100]:
geolocator = Nominatim(user_agent="myGeocoder", timeout=10)

def do_geocode(address):
    try:
        return geolocator.geocode(address)
    except GeocoderTimedOut:
        return do_geocode(address)

df_homicidios['coordenadas'] =df_homicidios['LUGAR_DEL_HECHO'].apply(do_geocode).apply(lambda loc: tuple(loc.point) if loc else None)

In [101]:
# Rellenamos los valores faltantes en 'pos x' y 'pos y' con los valores de 'coordenadas'
df_homicidios.loc[df_homicidios['pos x'].isna(), 'pos x'] = df_homicidios['coordenadas'].apply(lambda x: x[0] if x else None)
df_homicidios.loc[df_homicidios['pos y'].isna(), 'pos y'] = df_homicidios['coordenadas'].apply(lambda x: x[1] if x else None)
# Borramos la columna coordenadas una vez realizados los cambios

df_homicidios = df_homicidios.drop('coordenadas', axis=1)


In [102]:
# Reemplazamos los valores "." por 0 en 'pos x' y 'pos y'
df_homicidios['pos x'] = df_homicidios['pos x'].replace(".", 0)
df_homicidios['pos y'] = df_homicidios['pos y'].replace(".", 0)

In [103]:
df_homicidios['Anio'] = df_homicidios['FECHA_SINIESTRO'].dt.year

In [104]:
df_homicidios

,ID,ROL,VICTIMA,SEXO,EDAD,FECHA_FALLECIMIENTO,N_VICTIMAS,FECHA_SINIESTRO,HORA_SINIESTRO,LUGAR_DEL_HECHO,TIPO_DE_CALLE,COMUNA,pos x,pos y,PARTICIPANTES,ACUSADO,semestre,Anio
0,1,Conductor,Moto,Masculino,19,2016-01-01 00:00:00,1,2016-01-01,04:00:00,Av Piedra Buena Y Av Fernandez De La Cruz,Avenida,8,-58.47533969,-34.68757022,Moto-Auto,Auto,1,2016
1,2,Conductor,Auto,Masculino,70,2016-01-02 00:00:00,1,2016-01-02,01:15:00,Av Gral Paz Y Av De Los Corrales,Gral Paz,9,-58.50877521,-34.66977709,Auto-Pasajeros,Pasajeros,1,2016
2,3,Conductor,Moto,Masculino,30,2016-01-03 00:00:00,1,2016-01-03,07:00:00,Av Entre Rios 2034,Avenida,1,-58.39040293,-34.63189362,Moto-Auto,Auto,1,2016
4,5,Conductor,Moto,Masculino,29,2016-02-01 00:00:00,1,2016-01-21,05:20:00,Av San Juan Y Presidente Luis Saenz Peña,Avenida,1,-58.38718297,-34.62246630,Moto-Pasajeros,Pasajeros,1,2016
5,8,Conductor,Moto,Masculino,30,2016-01-24 00:00:00,1,2016-01-24,18:30:00,Av 27 De Febrero Y Av Escalada,Avenida,8,-58.44451316,-34.68475866,Moto-Objeto Fijo,Objeto Fijo,1,2016
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
712,712,Peaton,Peaton,Femenino,50,2021-12-12 00:00:00,1,2021-12-12,06:20:00,Av. Rivadavia Y Av. Pueyrredon,Avenida,3,-58.40596860,-34.61011987,Peaton-Auto,Auto,2,2021
713,713,Pasajero_Acompañante,Moto,Femenino,18,2021-12-18 00:00:00,1,2021-12-13,17:10:00,Av. Riestra Y Mom,Avenida,7,-58.43353773,-34.64561636,Moto-Auto,Auto,2,2021
714,714,Pasajero_Acompañante,Moto,Femenino,43,2021-12-20 00:00:00,1,2021-12-20,01:10:00,Au Dellepiane Y Lacarra,Autopista,9,-58.46739825,-34.65117757,Moto-Auto,Auto,2,2021
715,715,Conductor,Moto,Masculino,27,2022-01-02 00:00:00,1,2021-12-30,00:43:00,Av. Gaona Y Terrada,Avenida,11,-58.47293407,-34.61984745,Moto-Cargas,Cargas,2,2021


# Comenzamos la búsqueda del KPI numero 1 agrupando las columnas necesarias del ETL


In [105]:
df_homicidios_sum = df_homicidios.groupby(['Anio', 'semestre'])['N_VICTIMAS'].sum().reset_index()

In [106]:
df_homicidios_sum

,Anio,semestre,N_VICTIMAS
0,2016,1,62
1,2016,2,75
2,2017,1,78
3,2017,2,78
4,2018,1,76
5,2018,2,83
6,2019,1,56
7,2019,2,47
8,2020,1,31
9,2020,2,56


### Definimos la tasa de accidentes por poblacion (KPI 1)

In [107]:
df_poblacion = pd.read_csv('poblacionCABA.csv')

In [108]:
# Elimnamos las columnas innecesarias
def clean_data(df_poblacion):
    df_poblacion = df_poblacion.drop(columns=['Unnamed: 0'])
    df_poblacion = df_poblacion.drop(columns=['Poblacion 2010'])
    df_poblacion = df_poblacion.drop(columns=['Crecimiento 2010-2022'])
    return df_poblacion

df_poblacion = clean_data(df_poblacion.copy())
df_poblacion

,Comuna,Poblacion 2016,Poblacion 2017,Poblacion 2018,Poblacion 2019,Poblacion 2020,Poblacion 2021
0,13,247686,250522,253391,256292,259227,262195
1,14,237203,239129,241071,243028,245001,246990
2,12,217446,220476,223550,226666,229825,233028
3,4,224495,225554,226618,227687,228761,229840
4,1,214398,215850,217313,218785,220267,221760
5,7,218707,218395,218083,217772,217461,217151
6,8,195839,197311,198793,200287,201793,203309
7,6,189418,191738,194087,196464,198871,201307
8,11,196538,197678,198825,199979,201139,202306
9,15,190078,191359,192648,193946,195252,196567


In [109]:
poblacion_sum = df_poblacion[['Poblacion 2016', 'Poblacion 2017', 'Poblacion 2018', 'Poblacion 2019', 'Poblacion 2020', 'Poblacion 2021']].sum()

In [110]:
poblacion_sum_df = poblacion_sum.to_frame().reset_index()
poblacion_sum_df.columns = ['Anio', 'Total Poblacion']

In [111]:
poblacion_sum_df

,Anio,Total Poblacion
0,Poblacion 2016,3002178
1,Poblacion 2017,3021444
2,Poblacion 2018,3040889
3,Poblacion 2019,3060508
4,Poblacion 2020,3080306
5,Poblacion 2021,3100282


In [112]:
poblacion_sum_df['Anio'] = poblacion_sum_df['Anio'].str.replace('Poblacion', '', case=False)

In [113]:
poblacion_sum_df

,Anio,Total Poblacion
0,2016,3002178
1,2017,3021444
2,2018,3040889
3,2019,3060508
4,2020,3080306
5,2021,3100282


In [114]:
def clean_data(df_homicidios_sum):
    # Cambiamos el tipo de dato de "Anio"
    df_homicidios_sum = df_homicidios_sum.astype({'Anio': 'int64'})
    return df_homicidios_sum

df_homicidios_sum = clean_data(df_homicidios_sum.copy())
df_homicidios_sum

,Anio,semestre,N_VICTIMAS
0,2016,1,62
1,2016,2,75
2,2017,1,78
3,2017,2,78
4,2018,1,76
5,2018,2,83
6,2019,1,56
7,2019,2,47
8,2020,1,31
9,2020,2,56


In [115]:
def clean_data(df_homicidios_sum):
    # Cambiamos el tipo de dato de Anio
    df_homicidios_sum = df_homicidios_sum.astype({'Anio': 'int64'})
    return df_homicidios_sum

poblacion_sum_df = clean_data(poblacion_sum_df.copy())
poblacion_sum_df.head(2)

,Anio,Total Poblacion
0,2016,3002178
1,2017,3021444


In [116]:
df_homicidios_victimas = df_homicidios_sum.merge(poblacion_sum_df, on='Anio')

In [117]:
df_homicidios_victimas

,Anio,semestre,N_VICTIMAS,Total Poblacion
0,2016,1,62,3002178
1,2016,2,75,3002178
2,2017,1,78,3021444
3,2017,2,78,3021444
4,2018,1,76,3040889
5,2018,2,83,3040889
6,2019,1,56,3060508
7,2019,2,47,3060508
8,2020,1,31,3080306
9,2020,2,56,3080306


# Primer KPI

In [118]:
df_homicidios_victimas['tasa'] = (df_homicidios_victimas['N_VICTIMAS'] / df_homicidios_victimas['Total Poblacion']) * 100000

In [119]:
df_homicidios_victimas

,Anio,semestre,N_VICTIMAS,Total Poblacion,tasa
0,2016,1,62,3002178,2.065167
1,2016,2,75,3002178,2.498186
2,2017,1,78,3021444,2.581547
3,2017,2,78,3021444,2.581547
4,2018,1,76,3040889,2.499269
5,2018,2,83,3040889,2.729465
6,2019,1,56,3060508,1.829762
7,2019,2,47,3060508,1.535693
8,2020,1,31,3080306,1.006394
9,2020,2,56,3080306,1.818001


In [120]:
df_homicidios_victimas['Tasa Anterior'] = df_homicidios_victimas['tasa'].shift(periods=1, fill_value=0)

In [121]:
df_homicidios_victimas

,Anio,semestre,N_VICTIMAS,Total Poblacion,tasa,Tasa Anterior
0,2016,1,62,3002178,2.065167,0.000000
1,2016,2,75,3002178,2.498186,2.065167
2,2017,1,78,3021444,2.581547,2.498186
3,2017,2,78,3021444,2.581547,2.581547
4,2018,1,76,3040889,2.499269,2.581547
5,2018,2,83,3040889,2.729465,2.499269
6,2019,1,56,3060508,1.829762,2.729465
7,2019,2,47,3060508,1.535693,1.829762
8,2020,1,31,3080306,1.006394,1.535693
9,2020,2,56,3080306,1.818001,1.006394


In [122]:
df_homicidios_victimas['VARIACION_TASA'] = (df_homicidios_victimas['tasa'] - df_homicidios_victimas['Tasa Anterior']) / df_homicidios_victimas['Tasa Anterior'] * 100

In [123]:
df_homicidios_victimas

,Anio,semestre,N_VICTIMAS,Total Poblacion,tasa,Tasa Anterior,VARIACION_TASA
0,2016,1,62,3002178,2.065167,0.000000,inf
1,2016,2,75,3002178,2.498186,2.065167,20.967742
2,2017,1,78,3021444,2.581547,2.498186,3.336852
3,2017,2,78,3021444,2.581547,2.581547,0.000000
4,2018,1,76,3040889,2.499269,2.581547,-3.187158
5,2018,2,83,3040889,2.729465,2.499269,9.210526
6,2019,1,56,3060508,1.829762,2.729465,-32.962628
7,2019,2,47,3060508,1.535693,1.829762,-16.071429
8,2020,1,31,3080306,1.006394,1.535693,-34.466480
9,2020,2,56,3080306,1.818001,1.006394,80.645161


In [124]:
df_homicidios_victimas['VARIACION_TASA'] = df_homicidios_victimas['VARIACION_TASA'].replace([np.inf, -np.inf], 0)

In [125]:

def clean_data(df_homicidios_victimas):
    # Redondeamos a 2 decimales
    df_homicidios_victimas = df_homicidios_victimas.round({'VARIACION_TASA': 2})
    return df_homicidios_victimas

df_homicidios_victimas = clean_data(df_homicidios_victimas.copy())
df_homicidios_victimas.head(2)

,Anio,semestre,N_VICTIMAS,Total Poblacion,tasa,Tasa Anterior,VARIACION_TASA
0,2016,1,62,3002178,2.065167,0.000000,0.00
1,2016,2,75,3002178,2.498186,2.065167,20.97


# KPI 2

In [126]:
df_homicidios_moto = df_homicidios[df_homicidios['VICTIMA'] == 'Moto']

In [127]:
df_homicidios_moto

,ID,ROL,VICTIMA,SEXO,EDAD,FECHA_FALLECIMIENTO,N_VICTIMAS,FECHA_SINIESTRO,HORA_SINIESTRO,LUGAR_DEL_HECHO,TIPO_DE_CALLE,COMUNA,pos x,pos y,PARTICIPANTES,ACUSADO,semestre,Anio
0,1,Conductor,Moto,Masculino,19,2016-01-01 00:00:00,1,2016-01-01,04:00:00,Av Piedra Buena Y Av Fernandez De La Cruz,Avenida,8,-58.47533969,-34.68757022,Moto-Auto,Auto,1,2016
2,3,Conductor,Moto,Masculino,30,2016-01-03 00:00:00,1,2016-01-03,07:00:00,Av Entre Rios 2034,Avenida,1,-58.39040293,-34.63189362,Moto-Auto,Auto,1,2016
4,5,Conductor,Moto,Masculino,29,2016-02-01 00:00:00,1,2016-01-21,05:20:00,Av San Juan Y Presidente Luis Saenz Peña,Avenida,1,-58.38718297,-34.62246630,Moto-Pasajeros,Pasajeros,1,2016
5,8,Conductor,Moto,Masculino,30,2016-01-24 00:00:00,1,2016-01-24,18:30:00,Av 27 De Febrero Y Av Escalada,Avenida,8,-58.44451316,-34.68475866,Moto-Objeto Fijo,Objeto Fijo,1,2016
6,9,Pasajero_Acompañante,Moto,Masculino,29,2016-01-26 00:00:00,1,2016-01-24,19:10:00,Nogoya Y Joaquin V. Gonzales,Calle,11,-58.50095869,-34.60825440,Moto-Auto,Auto,1,2016
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
709,709,Conductor,Moto,Masculino,41,2021-12-11 00:00:00,1,2021-12-02,01:10:00,Av. Gaona 3655,Avenida,11,-58.47633683,-34.62140594,Moto-Auto,Auto,2,2021
711,711,Conductor,Moto,Masculino,24,2021-12-11 00:00:00,1,2021-12-11,23:00:00,Baigorria Y Victor Hugo,Calle,10,-58.51989389,-34.62284918,Moto-Auto,Auto,2,2021
713,713,Pasajero_Acompañante,Moto,Femenino,18,2021-12-18 00:00:00,1,2021-12-13,17:10:00,Av. Riestra Y Mom,Avenida,7,-58.43353773,-34.64561636,Moto-Auto,Auto,2,2021
714,714,Pasajero_Acompañante,Moto,Femenino,43,2021-12-20 00:00:00,1,2021-12-20,01:10:00,Au Dellepiane Y Lacarra,Autopista,9,-58.46739825,-34.65117757,Moto-Auto,Auto,2,2021


In [128]:
df_homicidios_moto = df_homicidios_moto.groupby(['Anio'])['N_VICTIMAS'].sum().reset_index()

In [129]:
df_homicidios_moto

,Anio,N_VICTIMAS
0,2016,63
1,2017,59
2,2018,61
3,2019,48
4,2020,32
5,2021,46


In [130]:
df_homicidios_moto['N_VICTIMAS_ANTERIOR'] = df_homicidios_moto['N_VICTIMAS'].shift(periods=1, fill_value=0)

In [131]:
df_homicidios_moto

,Anio,N_VICTIMAS,N_VICTIMAS_ANTERIOR
0,2016,63,0
1,2017,59,63
2,2018,61,59
3,2019,48,61
4,2020,32,48
5,2021,46,32


In [132]:
df_homicidios_moto['VARIACION_VICTIMAS_MOTO'] = (df_homicidios_moto['N_VICTIMAS'] - df_homicidios_moto['N_VICTIMAS_ANTERIOR']) / df_homicidios_moto['N_VICTIMAS_ANTERIOR'] * 100
df_homicidios_moto

,Anio,N_VICTIMAS,N_VICTIMAS_ANTERIOR,VARIACION_VICTIMAS_MOTO
0,2016,63,0,inf
1,2017,59,63,-6.349206
2,2018,61,59,3.389831
3,2019,48,61,-21.311475
4,2020,32,48,-33.333333
5,2021,46,32,43.750000


In [133]:
df_homicidios_moto['VARIACION_VICTIMAS_MOTO'] = df_homicidios_moto['VARIACION_VICTIMAS_MOTO'].replace([np.inf, -np.inf], 0)

In [134]:

def clean_data(df_homicidios_moto):
    # Redondeamos la variacion a 2 decimales
    df_homicidios_moto = df_homicidios_moto.round({'VARIACION_VICTIMAS_MOTO': 2})
    return df_homicidios_moto

df_homicidios_moto = clean_data(df_homicidios_moto.copy())
df_homicidios_moto.head()

,Anio,N_VICTIMAS,N_VICTIMAS_ANTERIOR,VARIACION_VICTIMAS_MOTO
0,2016,63,0,0.00
1,2017,59,63,-6.35
2,2018,61,59,3.39
3,2019,48,61,-21.31
4,2020,32,48,-33.33


Guardamos nuestros archivos

In [135]:
df_homicidios_moto.to_csv('Tasa_homicidios_moto.csv')

In [136]:
df_homicidios_victimas.to_csv('Tasa_homicidios.csv')

In [137]:
df_homicidios.to_csv('homicidiosfinal.csv')